<a href="https://colab.research.google.com/github/cxbxmxcx/Practical_AI_on_GCP/blob/master/Chapter_9_Build_Nutritionist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

import numpy as np
import os
import time
from PIL import Image
import pickle

Download Recipe Data

In [ ]:
data_folder = 'data'
recipes_zip = tf.keras.utils.get_file('recipes.zip',                                     
                                          origin = 'https://www.dropbox.com/s/i1hvs96mnahozq0/Recipes5k.zip?dl=1',
                                          extract = True)
print(recipes_zip)
data_folder = os.path.dirname(recipes_zip)
os.remove(recipes_zip)
print(data_folder)

164929536/164925728 [==============================] - 13s 0us/step
/root/.keras/datasets/recipes.zip
/root/.keras/datasets


Setup Folder Paths

In [ ]:
!dir /root/.keras/datasets
data_folder = data_folder + '/Recipes5k/'
annotations_folder = data_folder + 'annotations/'
images_folder = data_folder + 'images/'
print(annotations_folder)
print(images_folder)

Recipes5k
/root/.keras/datasets/Recipes5k/annotations/
/root/.keras/datasets/Recipes5k/images/


In [ ]:
%ls /root/.keras/datasets/Recipes5k/images/

apple_pie/            dumplings/                omelette/
baby_back_ribs/       edamame/                  onion_rings/
baklava/              eggs_benedict/            oysters/
beef_carpaccio/       escargots/                pad_thai/
beef_tacos/           falafel/                  paella/
beef_tartare/         filet_mignon/             pancakes/
beet_salad/           fish_and_chips/           panna_cotta/
beignets/             foie_gras/                peking_duck/
bibimbap/             french_fries/             pho/
bread_pudding/        french_onion_soup/        pizza/
breakfast_burrito/    french_toast/             pork_chop/
bruschetta/           fried_calamari/           poutine/
caesar_salad/         fried_rice/               prime_rib/
cannoli/              frozen_yogurt/            pulled_pork_sandwich/
caprese_salad/        garlic_bread/             ramen/
carrot_cake/          gnocchi/                  ravioli/
ceviche/              greek_salad/              red_velvet_cake/


Extra Imports

In [ ]:
from fastprogress.fastprogress import master_bar, progress_bar
from IPython.display import Image
from os import listdir
from pickle import dump

Setup Convnet Application

In [ ]:
use_NAS = False
if use_NAS:
  IMG_SIZE = 224 # 299 for Inception, 224 for NASNet
  IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
else:
  IMG_SIZE = 299 # 299 for Inception, 224 for NASNet
  IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)


In [ ]:
def load_image(image_path):
  img = tf.io.read_file(image_path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
  if use_NAS:
    img = tf.keras.applications.nasnet.preprocess_input(img)
  else:
    img = tf.keras.applications.inception_v3.preprocess_input(img)
  return img, image_path

In [ ]:
foods_txt = tf.keras.utils.get_file('foods.txt',
                        origin = 'https://www.dropbox.com/s/xyukyq62g98dx24/foods_cat.txt?dl=1')

print(foods_txt)

8192/1968 [============================================================================================================================] - 0s 0us/step
/root/.keras/datasets/foods.txt


In [ ]:
def get_nutrient_array(fat, protein, carbs):
  nutrients = np.array([float(fat)*4, float(protein)*4, float(carbs)*4])  
  nutrients /= np.linalg.norm(nutrients)
  return nutrients

In [ ]:
def get_category_array(keto, carbs, health):
  return np.array([float(keto)-5, float(carbs)-5, float(health)-5])

In [ ]:
import csv

def get_food_nutrients(nutrient_file):
  foods = {}
  with open(foods_txt) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
      if line_count == 0:
        print(f'Column names are {", ".join(row)}')
        line_count += 1
      else:        
        categories = get_category_array(row[1],row[2],row[3])
        foods[row[0]] = categories
        line_count += 1
    print(f'Processed {line_count} lines.')
  return foods

In [ ]:
food_nutrients = get_food_nutrients(foods_txt)
print(food_nutrients)

Column names are name, keto, carbs, health
Processed 102 lines.
{'apple_pie': array([-4.,  5., -4.]), 'baby_back_ribs': array([ 4., -2.,  1.]), 'baklava': array([-4.,  5., -1.]), 'beef_carpaccio': array([ 5., -4.,  3.]), 'beef_tacos': array([-3., -1.,  2.]), 'beef_tartare': array([ 5., -4.,  3.]), 'beet_salad': array([-3.,  3.,  5.]), 'beignets': array([-4.,  5., -1.]), 'bibimbap': array([-2.,  3.,  0.]), 'bread_pudding': array([-4.,  3.,  0.]), 'breakfast_burrito': array([-2.,  2.,  3.]), 'bruschetta': array([-1.,  2.,  4.]), 'caesar_salad': array([ 1., -2.,  5.]), 'cannoli': array([-4.,  5., -1.]), 'caprese_salad': array([0., 0., 5.]), 'carrot_cake': array([-4.,  5.,  1.]), 'ceviche': array([ 4., -4.,  4.]), 'cheesecake': array([-4.,  4., -2.]), 'cheese_plate': array([ 4., -4.,  2.]), 'chicken_curry': array([ 4., -3.,  4.]), 'chicken quesadilla': array([1., 0., 3.]), 'chicken_wings': array([ 4., -3.,  3.]), 'chocolate_cake': array([-4.,  5., -3.]), 'chocolate_ice_cream': array([-4., 

In [ ]:
def load_images(food_w_nutrients, directory):
  X = []
  Y = []
  i=0
  mb = master_bar(listdir(directory))
  for food_group in mb: 
    try:
      for pic in progress_bar(listdir(directory + food_group),
                              parent=mb, comment='food = ' + food_group):
        filename = directory + food_group + '/' + pic
        image, img_path = load_image(filename)
        if i < 5:
          print(img_path)
          i+=1
        Y.append(food_w_nutrients[food_group])
        X.append(image)
    except:
      continue
  return X,Y


In [ ]:
X, Y = load_images(food_nutrients, images_folder)
print(len(X), len(Y))

/root/.keras/datasets/Recipes5k/images/beef_carpaccio/1_beef_carpaccio_crostinis_hostedLargeUrl.jpg
/root/.keras/datasets/Recipes5k/images/beef_carpaccio/19_nigerian_suya_&_nut_butters_hostedLargeUrl.jpg
/root/.keras/datasets/Recipes5k/images/beef_carpaccio/16_carpaccio_di_manzo,_pinzimonio,_e_ricotta_salata_(beef_carpaccio,_pinzimonio,_and_salted_cheese)_hostedLargeUrl.jpg
/root/.keras/datasets/Recipes5k/images/beef_carpaccio/9_painted_chef's_classic_beef_carpaccio_hostedLargeUrl.jpg
/root/.keras/datasets/Recipes5k/images/beef_carpaccio/8_beef_tenderloin_carpaccio_hostedLargeUrl.jpg
4776 4776


In [ ]:
tf.keras.backend.clear_session()

if use_NAS:
  # Create the base model from the pre-trained model 
  base_model = tf.keras.applications.NASNetMobile(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights='imagenet')
else:
  # Create the base model from the pre-trained model 
  base_model = tf.keras.applications.InceptionResNetV2(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights='imagenet')

219062272/219055592 [==============================] - 1s 0us/step


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X, Y))
dataset
batches = dataset.batch(64)

In [ ]:
for image_batch, label_batch in batches.take(1):
   pass

image_batch.shape
train_size = int(len(X)*.8)
test_size = int(len(X)*.2)

batches = batches.shuffle(test_size)
train_dataset = batches.take(train_size)
test_dataset = batches.skip(train_size)
test_dataset = test_dataset.take(test_size)

In [ ]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(64, 8, 8, 1536)


In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
if use_NAS:
  fine_tune_at = 100
else:
  fine_tune_at = 550

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

base_model.summary()

Number of layers in the base model:  780
Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
_______________________________________

Add Regression Head

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(64, 1536)


In [ ]:
prediction_layer = tf.keras.layers.Dense(3)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(64, 3)


In [ ]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Nadam(lr=base_learning_rate),
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse', 'accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 8, 8, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 4611      
Total params: 54,341,347
Trainable params: 30,664,483
Non-trainable params: 23,676,864
_________________________________________________________________


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

folder = '/content/gdrive/My Drive/Models'
if os.path.isdir(folder) == False:
  os.makedirs(folder)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = folder + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

In [ ]:
history = model.fit(batches,epochs=25, callbacks=[cp_callback])

Epoch 1/25
75/75 [==============================] - 38s 500ms/step - loss: 2.6666 - mae: 2.6666 - mse: 10.1546 - accuracy: 0.3428
Epoch 2/25
75/75 [==============================] - 37s 487ms/step - loss: 2.3121 - mae: 2.3121 - mse: 8.4882 - accuracy: 0.4169
Epoch 3/25
75/75 [==============================] - 37s 487ms/step - loss: 2.0193 - mae: 2.0193 - mse: 7.0606 - accuracy: 0.4943
Epoch 4/25
75/75 [==============================] - 36s 486ms/step - loss: 1.8176 - mae: 1.8176 - mse: 6.0534 - accuracy: 0.5586
Epoch 5/25
75/75 [==============================] - ETA: 0s - loss: 1.6107 - mae: 1.6107 - mse: 4.9274 - accuracy: 0.6087
Epoch 00005: saving model to /content/gdrive/My Drive/Models/cp-0005.ckpt
75/75 [==============================] - 42s 556ms/step - loss: 1.6107 - mae: 1.6107 - mse: 4.9274 - accuracy: 0.6087
Epoch 6/25
75/75 [==============================] - 36s 486ms/step - loss: 1.3488 - mae: 1.3488 - mse: 3.5469 - accuracy: 0.6520
Epoch 7/25
75/75 [======================

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
mae = history.history['mae']
mse = history.history['mse']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Loss')
plt.legend(loc='upper right')
plt.ylabel('MAE')
plt.ylim([0,5.0])
plt.title('Training Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
def get_test_images():
  directory = '/content/'
  images = []
  for file in listdir(directory):    
    if file.endswith(".jpg"):
      images.append(file)
  return images

images = get_test_images()
print(images)

In [ ]:
#@title Image Prediction { run: "auto", vertical-output: true, display-mode: "form" }

image_idx = 42 #@param {type:"slider", min:0, max:100, step:1}
cnt = len(images)
if  cnt > 0:
  image_idx = image_idx if image_idx < cnt else cnt - 1
  image = images[image_idx]
  x, _ = load_image(image)

  img = x[np.newaxis, ...]
  predict = model.predict(img)
  print(predict+5)
  print(image_idx,image)
  plt.imshow(x)